In [10]:
import numpy as np
import tensorflow as tf
import random, math

In [2]:
## Hyper-Parameters
n = 1000 # number of samples
d = 10  # input dimension
k = 5   # hidden layer size
kappa = 2 
T = 100 # num of epochs
lr = 0.02
batch_size = 20

In [3]:
## Activation Function
def phi(h):
    return h**2 if h > 0 else 0
phi = np.vectorize(phi)

In [4]:
## Data and Params Generation

gauss_mat = np.random.normal(0.0, 0.1 , (d,k))
U, V = np.linalg.qr(gauss_mat) 

### TODO: U,V both are orthogonal and come from different gaussian matrices

diag = []
v_gt = []
v_choice = [1,-1]
for iter in range(k):
    diag.append(1+1.*iter*(kappa-1)/(k-1))
    v_gt.append(random.choice(v_choice))
    
Sigma = np.diag(diag)
W_gt = np.dot(np.dot(U, Sigma), np.transpose(V))
v_gt = np.asarray(v_gt)
train_x = []
train_y = []
for iter in range(n):
    train_x.append(np.random.normal(0.0,1.,d))
    train_y.append(np.dot(phi(np.dot(train_x[iter], W_gt)),v_gt))
train_x = np.asarray(train_x)
train_y = np.transpose(np.asarray(train_y))

In [54]:
def outer3(a,b,c):
    A = np.outer(a,b)
    B = []
    for third in c:
        B.append(A*third)
    return np.asarray(B)

def outer4(a,b,c,d):
    A = outer3(a,b,c)
    B = []
    for fourth in d:
        B.append(A*fourth)
    return np.asarray(B)

def outer3I(x):
    return outer3(x,x,x)

def outer4I(x):
    return outer4(x,x,x,x)

def specOuterI(x):
    d = len(x)
    iden = np.identity(d)
    final = np.zeros([d,d,d])
    for i in range(d):
        final += outer3(x, iden[i], iden[i]) + outer3(iden[i], x, iden[i])+ outer3(iden[i], iden[i],x)
    return final

def specOuterMat(M):
    d = np.shape(M)[0]
    ## TODO
    
    

In [58]:
def prob(x):
    return math.pow(math.e, -0.5*np.dot(np.transpose(x),x))/np.sqrt(math.pow(2*math.pi, len(x)))

def getM1(X, y):
    M1 = np.zeros(d)
    for iter in range(np.shape(X)[0]):
        M1 += y[iter]*X[iter]*prob(X[iter])
    return M1

def getM2(X, y):
    M2 = np.zeros([d,d])
    for iter in range(np.shape(X)[0]):
        M2 += y[iter]*(np.outer(X[iter], X[iter]) - np.identity(d))*prob(X[iter])
    return M2

def getM3(X, y):
    M3 = np.zeros([d,d,d])
    for iter in range(np.shape(X)[0]):
        M3 += y[iter]*(outer3I(X[iter]) - specOuterI(X[iter]) )*prob(X[iter])
    return M3

def getM4(X, y):
    M4 = np.zeros([d,d,d,d])
    for iter in range(np.shape(X)[0]):
        M3 += y[iter]*(outer4I(X[iter]) - specOuterMat(np.outer(X[iter], X[iter])) + specOuterMat(np.identity(d)) )*prob(X[iter])
    return M3



In [ ]:
def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    """
    h    = tf.square(tf.nn.relu((tf.matmul(X, w_1))))
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

In [ ]:
X = tf.placeholder("float", shape=[None, d])
y = tf.placeholder("float", shape=[None, 1])

# Weight initializations
w_1 = init_weights((d, k))
w_2 = init_weights((k, 1))

# Forward propagation
yhat  = forwardprop(X, w_1, w_2)

# Backward propagation
cost = tf.losses.mean_squared_error(y, yhat)
updates = tf.train.GradientDescentOptimizer(lr).minimize(cost)

# Run SGD
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
init = tf.global_variables_initializer()
sess.run(init)
epsilon = 1e-4
for epoch in range(T):
    # Train with each example
    i = 0
    for iter in range(int(n/batch_size)):
        sess.run(updates, feed_dict={X: train_x[i: i + batch_size], y: train_y[i: i + batch_size].reshape(batch_size,1)})
        i = (i + batch_size)%n
    train_accuracy = np.mean((train_y - sess.run(yhat, feed_dict={X:train_x, y:train_y.reshape(n,1)})) <= epsilon)
    print("Epoch = ", epoch+1," train Acc: ",100.*train_accuracy )